In [1]:
!pip install -q langchain_community langchain openai
!pip install -q huggingface_hub transformers sentencepiece accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [3]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=openai_api_key,
             model_name='text-davinci-003',
             temperature=0.9)

<ipython-input-4-e9dba7d4222a>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=openai_api_key,


### Listing

In [ ]:
task = " List Three popular Arabian plates."

print( llm(task) )

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts.prompt import PromptTemplate

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()


In [ ]:
prompt = PromptTemplate(
    template = "List Three popular { type } plates.\n{format_instructions}",
    input_variables = ["type"],
    partial_variables = {"format_instructions": format_instructions}

)

In [ ]:
demo_message = prompt.format(type="Egyptian")
# print(llm(demo_message))

response = llm(demo_message)
output = output_parser.parse(response)
print(output)

print(type(output)) # list

### DateTime

In [ ]:
format_instructions = "replay with format DAY-MONTH-YEAR like 28-08-2003"

prompt = PromptTemplate(
    template=" when was {person} born ?\n{format_instructions}",
    input_variables=["person", "format_instructions"]
)

response = llm ( prompt.format(person="Geoffrey Hinton", format_instructions=format_instructions))
print(response)

### PyDantic

In [12]:
from pydantic import BaseModel, Field
from typing import List
from langchain.output_parsers import PydanticOutputParser

In [13]:
class plate(BaseModel):
  plate_name:str=Field(description="name of the plate")
  ingredients:List[str] = Field(description="ingredients of the plate")

parser = PydanticOutputParser(pydantic_object=plate)


In [14]:
format_instructions = parser.get_format_instructions()
prompt = PromptTemplate(
    template = "Answer The Query.\n{plate_name}\n{format_instructions}",
    input_variables = ["plate_name", "format_instructions"],
    partial_variables = {"format_instructions": format_instructions}
)

In [ ]:
plate_name = "shawrema"
prompt = prompt.format(plate_name=plate_name)
response = llm ( prompt )
# print( response )

output = parser.parse(response)
print(output)

In [ ]:
print(output.plate_name)
print(output.ingredients)